<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/risk_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
code = '090150'

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime

class Stock:
  def __init__(self, code_name, page:int=26):
    self.code_name = code_name
    self.page = page
    self.df = self.do_it()

  def page_to_df(self, page):
    url = "https://finance.naver.com/item/sise_day.nhn?code=" + self.code_name + "&page=" + str(page)
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}
    result = requests.get(url, headers = headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    tr = bs_obj.find_all("tr", {'onmouseover':'mouseOver(this)', 'onmouseout':"mouseOut(this)"})
    data_dict = {}
    for i in range(10):
      try: 
        spans = tr[i].find_all('span')
        lines = [span.get_text() for span in spans]
        lines = list(map(lambda x: x.replace(',',''), lines))

        objdate = datetime.datetime.strptime(lines[0], '%Y.%m.%d')
        data_dict[objdate] = lines[1:]
      except:
        break
      
    df = pd.DataFrame.from_dict(data_dict).transpose()
    df.columns = ['Close','_','Open',
                        'High','Low','Volume']
    df = df.drop(columns="_")

    return df

  def do_it(self):
    df = self.page_to_df(1)
    # max 9999
    # 1 page = 10 days
    for i in range(2, self.page): 
      try: 
        if (any(self.page_to_df(i).index != self.page_to_df(i-1).index)):
          df = df.append(self.page_to_df(i))
          print('.', end = '')
          if (i%100==0):
            print(i)
        else:
          break
      except:
        df = df.append(self.page_to_df(i))
        break
    df = df.sort_index(axis = 0) 
    for i in ['Close', 'Open', 'High', 'Low', 'Volume']:
      df[i] = df[i].astype(int)
    return df

  def get_df(self):
    return self.df

In [3]:
# 1 page = 10 days
stock = Stock(code, 26)

........................

In [4]:
market = Stock

In [5]:
df = stock.get_df()

In [6]:
df

,Close,Open,High,Low,Volume
2020-03-23,1870,1760,1950,1570,139573
2020-03-24,1870,1870,1915,1735,29568
2020-03-25,2010,1950,2015,1900,47413
2020-03-26,2030,2010,2100,1935,87665
2020-03-27,2085,2050,2185,1975,76423
...,...,...,...,...,...
2021-03-22,4400,4385,4500,4360,34722
2021-03-23,4370,4400,4470,4330,33945
2021-03-24,4250,4370,4370,4200,26780
2021-03-25,4100,4245,4245,4100,50346


In [7]:
change = df['Close'][1:].values - df['Close'][:-1].values

In [8]:
#normailization
norm = np.linalg.norm(change)
norm_chage = np.round(change/norm, 16)
print(norm_chage)

[ 0.          0.06958052  0.00994007  0.0273352   0.02236517  0.0273352
 -0.02485019 -0.00248502  0.01242509  0.03230524  0.08946067  0.01739513
  0.05715543  0.01491011 -0.07455056  0.01988015  0.03230524  0.01491011
  0.         -0.0397603   0.00745506  0.00497004 -0.00745506  0.02485019
  0.07455056 -0.04224532 -0.01739513  0.00248502 -0.01739513  0.04970037
 -0.00248502 -0.02236517  0.00497004 -0.0397603   0.00497004  0.05467041
  0.01988015  0.          0.01242509 -0.01988015  0.00745506 -0.00248502
 -0.01739513  0.0273352   0.05964045  0.05467041  0.01739513 -0.01739513
 -0.02485019  0.03727528 -0.01242509  0.01491011  0.00745506  0.00994007
 -0.05467041 -0.07952059  0.02485019  0.          0.02982022 -0.00745506
 -0.05964045  0.0397603  -0.01242509 -0.00745506  0.04970037 -0.01988015
  0.0273352  -0.00994007  0.00994007  0.00248502  0.05218539 -0.04473033
 -0.00994007  0.01242509  0.         -0.00994007 -0.00745506  0.00745506
  0.          0.01988015  0.04970037  0.00745506 -0.

In [9]:
print(code)
print('Mean: ', np.mean(norm_chage))
print('Std: ', np.std(norm_chage))
print('Sharp: ', np.mean(norm_chage)/np.std(norm_chage))
print('Beta: ', )

090150
Mean:  0.004520937385138552
Std:  0.06321095935190171
Sharp:  0.07152141703735333
